## Step 1: Load the dataset

In [ ]:
import os
import librosa
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, plot_confusion_matrix
import matplotlib.pyplot as plt

data_dir = './data/genres_original'  # Path to dataset directory
genres = os.listdir(data_dir)  # List of genre names

dataset = []  # List to store MFCCs and genre labels
for genre in genres:
    genre_dir = os.path.join(data_dir, genre)  # Path to genre directory
    for filename in os.listdir(genre_dir):
        songname = os.path.join(genre_dir, filename)  # Path to song file
        try:
            y, sr = librosa.load(songname, duration=30)  # Load audio file with duration of 30 seconds
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Extract MFCCs
            dataset.append((mfcc, genre))  # Append MFCCs and genre label to dataset list
        except:
            print(f'Error loading {songname}. Skipping...')
            continue

/Users/drewwhite/Desktop/Programming/Projects/musical_journeys/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: UserWarning: PySoundFile failed. Trying audioread instead.
/Users/drewwhite/Desktop/Programming/Projects/musical_journeys/venv/lib/python3.7/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error loading ./data/genres_original/jazz/jazz.00054.wav. Skipping...


## Step 2: Split the dataset into training and testing sets

In [ ]:
X, y = zip(*dataset)  # Unzip the MFCCs and genre labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  # Split into train and test sets

## Step 3: Reshape and scale the input features

In [ ]:
# Find the maximum number of frames in the dataset
max_frames = max([x.shape[1] for x in X_train + X_test])

# Pad or truncate the MFCCs to have the same number of frames
X_train = np.array([np.pad(x, ((0, 0), (0, max_frames - x.shape[1])), mode='constant') for x in X_train])
X_test = np.array([np.pad(x, ((0, 0), (0, max_frames - x.shape[1])), mode='constant') for x in X_test])

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train.reshape(len(X_train), -1)).reshape(X_train.shape)
X_test = scaler.transform(X_test.reshape(len(X_test), -1)).reshape(X_test.shape)


## Step 4: Train and evaluate the model